In [1]:
import pandas as pd
import geopandas as gpd
import requests
import json
from bs4 import BeautifulSoup
import os
import matplotlib.pyplot as plt
import contextily as ctx

# Initial Fucking Around

In [ ]:
# Yelp Search Results
r = requests.get('https://www.yelp.com/search?find_desc=bars&find_loc=Los+Angeles%2C+CA+90049&ns=1')
r.ok

In [ ]:
#r.text

In [ ]:
soup = BeautifulSoup(r.text)
print(soup.prettify())

In [ ]:
# YELP URL of One page (could extract from businessurl)
r = requests.get('https://www.yelp.com/biz/the-wellesbourne-los-angeles?osq=bars')

In [ ]:
soup = BeautifulSoup(r.text)
print(soup.prettify())

In [ ]:
soup.find_all("application/ld+json")

In [ ]:
# Foursquare API
# API has tips for each venue, can pull up to 
import requests

fsq_id = '5a187743ccad6b307315e6fe'
url = "https://api.foursquare.com/v3/places/{}/tips".format(fsq_id)

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

# Cleaned Yelp Dataset

## Filter Business JSON for Philadelphia Area

In [2]:
# load data
biz = pd.read_json('/Users/harper/Documents/GitHub/yelp_dataset/yelp_academic_dataset_business.json',lines=True)

# create geodataframe
bizGdf = gpd.GeoDataFrame(biz, geometry=gpd.points_from_xy(biz.longitude, biz.latitude), crs='EPSG:4326')

# filter for Philadelphia metro area
bizPhil = bizGdf[(bizGdf.state == 'PA') | (bizGdf.state == 'NJ') | (bizGdf.state == 'DE')]

In [ ]:
##  FIGURE OUT HOW TO GET ALL ATTRIBUTES TO CHECK FOR GAY
print(bizPhil['attributes'][biz].keys())

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(20,20))
bizPhil.to_crs('EPSG: 3857').plot(ax=ax)
ctx.add_basemap(ax)

plt.savefig('images/yelp_philly.png', dpi=300)

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(20,20))
bizGdf.to_crs('EPSG: 3857').plot(ax=ax)
ctx.add_basemap(ax)

plt.savefig('images/yelp_country.png', dpi=300)

## Bring in reviews for Philadelphia area

<div class="alert alert-block alert-danger">
Reset kernel before running loop! Takes 30+ mins to run
</div>

In [3]:
# create jsonreader object
reader = pd.read_json('/Users/harper/Documents/GitHub/yelp_dataset/yelp_academic_dataset_review.json', 
                    orient = 'records', lines=True, chunksize=1000)

# initialize empty dataframe
colnames = ['review_id','user_id','business_id','stars','useful','funny','cool','text','date']
reviews = pd.DataFrame(columns = colnames)

# iterate through, filter for state, then add each chunk to the dataframe.
for chunk in reader:
    subset = chunk[chunk['business_id'].isin(bizPhil.business_id)]
    reviews = pd.concat([reviews, subset])



In [9]:
reviews.to_pickle('data/reviewsPhil.pandas')

In [10]:
reviews.to_csv('data/reviewsPhil.csv')

In [ ]:
# just for testing -- dont run
limit = 10 
for index,chunk in zip(range(limit), reader):
    subset = chunk[chunk['business_id'].isin(bizPhil.business_id)]
    reviews = pd.concat([reviews, subset])